In [ ]:
from notebook_logging import log_progress

In [ ]:
import gym
import time
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


"""
SFFF
FHFH
FFFH
HFFG
[0, x, x, x, 0, x, 1, x, 2, 1, 0, x, x, 3, 1, x]  <- our gamma .9
[0, 3, 3, 3, 0, 3, 0, 1, 3, 1, 0, 0, 2, 2, 1, 1]  <- GA best
[3, 3, 1, 1, x, x, 1, x, 1, 1, 1, x, x, 2, 1, x]
[1, 1, 0, x, 2, x, 1, x, 2, 1, 0, x, x, 3, 2, x]  <- gamma 1.0
[1, 1, 0, x, 0, x, 0, x, 3, 2, 1, x, x, 0, 2, x]  <- gamma .5

"""

gamma = 0.5
alpha = 0.01

def updateQ(Q, s_t, s_tn, a, R):
    """
    Q -> Q matrix
    s_tn -> new state
    R -> reward
    a -> action
    """
    a_max = np.argmax(Q[s_tn])
    Q[s_t, a] = (1 - alpha)*Q[s_t, a] + alpha * (R + gamma*Q[s_tn, a_max])
    return Q

def choose_policy(states):
    import numpy.random as rn
    #print(states)
    v_max = np.amax(states)
    #print(v_max)
    indici = np.arange(len(states))[states == v_max]
    #print(indici)
    rn.shuffle(indici)
    return indici[0]


Q = np.zeros((16, 4))

print("Q(t:0)")
print(Q)

In [ ]:
acc = 0;
Qs = []
Rs = []
Ts = []
#best_policy = [0, 3, 3, 3, 0, 3, 0, 1, 3, 1, 0, 0, 2, 2, 1, 1] 
from gym import wrappers
env = gym.make('FrozenLake-v0')
env = wrappers.Monitor(env, '/tmp/frozenlake-experiment-1', force=True)
for i_episode in log_progress(range(1000)):
    s_old = env.reset()

    for t in range(100000):
        #env.render()
        policy = choose_policy(Q[s_old])
        s_new, reward, done, info = env.step(policy)
        if done:
            Q = updateQ(Q, s_old, s_new, policy, reward - 0.5)
            Qs.append(Q)
            Rs.append(reward - 0.1)
            Ts.append(t)
            break	
        else:
            Q = updateQ(Q, s_old, s_new, policy, reward - 0.001)
            s_old = s_new


time.sleep(1)
env.close()
Rs = np.array(Rs)
Ts = np.array(Ts)

In [ ]:
len(Qs)

In [ ]:
fit = Rs / Ts
print("Rs %f Ts %f mFit %f" % (Rs.sum(), Ts.sum(), fit.mean()))
plt.plot(fit)

In [ ]:
print("Policy learned: {}".format(np.argmax(Q, axis=1)))